In [1]:
import charts
import numpy as np
import math

from scipy.special import lambertw

Server running in the folder c:\Users\u0098668\Documents\Modelica\DistrictHeating\Notebooks at 127.0.0.1:50976


In [2]:
def h1(T1, T2=50, hs=0.88, ha=1.97, Tg=10):
    return 1./2*(T1*(hs+ha)-T2*(ha-hs)-2*Tg*hs)/(T1-Tg)

def h2(T1, T2, hs=0.88, ha=1.97, Tg=10):
    return 1./2*(T1*(hs+ha)+T2*(ha+hs)-2*Tg*hs)/(T2-Tg)

def h1_lambert(T1, T2, td, hs=0.88, ha=1.97, Tg=10, lam=0.026, cp=4180, rho=1000, D=0.05):
    Ts = (T1+T2)/2
    Ta = (T1-T2)/2
    
    A = 1./4*D**2
    C = cp*rho*A
    b = 2*3.14*lam/C
    
    W = hs*hs*(Ts-Tg)*np.exp(-hs*b*td) + ha*ha*Ta*np.exp(-ha*b*td)
    z = - b*td*np.sqrt(W)/(2*np.sqrt(T1-Tg))
    print z
    
    return -2*lambertw(z,0).real / (b*td)
    

def expdecay(T, h, td, Tg=10, hs=0.88, ha=1.97, cp=4180, lam=0.026, rho=1000, D=0.05):
    A = 1./4*D**2
    C = cp*rho*A
    R = 1/(lam*2*3.14*h)
    tau = R*C
    return Tg + (T-Tg)*np.exp(-td/tau)

def expdecay_twostep(T, td, Tg=10):
    h = h1(T)
    T2 = expdecay(T, h, td, Tg)
    h = h1((T+T2)/2)
    return expdecay(T, h, td, Tg)

In [3]:
T1 = np.linspace(20, 70)
series = [{
    'name': 'Fixed T1',
    'data': np.array([T1, h1(T1)]).T
}]
charts.plot(series, show='inline')

In [4]:
td = np.linspace(60, 60*60*10, 3)
T1 = np.linspace(30, 70)

series = []
for t in td:
    series.append(
        dict(name=str(t/60), 
            data=np.array([T1, expdecay(70, h1(T1), t)]).T
        )
    )
    series.append(
        dict(
            name=str(t/60) + ' two step', 
            data=np.array([T1, expdecay_twostep(70, t)*np.ones(len(T1))]).T
        )
    )
    series.append(
        dict(
            name=str(t/60) + ' Lambert',
            data=np.array([T1, np.ones(len(T1))*expdecay(70, h1_lambert(70, 20, t), t)]).T
        )
    )

-0.00268809451387
-0.330085512441
-0.321342511551


In [5]:
charts.plot(series, options='decay', show='inline', save='decay.html')

In [6]:
def lambert_series(z, n):
    W = 0
    for i in range(1,n+1):
        W += math.pow(-i, i-1)/math.factorial(i) * math.pow(z,i)
    return W

In [7]:
series = []
ns = np.linspace(1,50,50)
for z in zs:
    res = []
    for n in ns:
        res.append(lambert_series(z, int(n)))
    series.append(dict(name=str(z), data=np.array([ns, res]).T))
    series.append(dict(name=str(z) + ' Real', data=np.array([ns, np.ones(len(ns))*lambertw(z).real]).T))
    


NameError: name 'zs' is not defined

In [ ]:
charts.plot(series, options='lambert-series', show='inline')

In [9]:
charts.default_settings.update(dict(show='inline'))

In [12]:
charts.plot([1,2,3])